In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 366 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 8.4 MB/s 
  Created wheel for openai: filename=openai-0.20.0-py3-none-any.whl size=54118 sha256=a487d953367ddd4ffa94ded87a73ec2373566bf57680078e48ee191b61a88be6
  Stored in directory: /root/.cache/pip/wheels/71/8d/9b/e28529ec53123e0279208f99148d4661232120d78cb866839b
Successfully built openai


In [1]:
!pip install transformers tinydb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 30.1 MB/s 
     |████████████████████████████████| 7.6 MB 58.9 MB/s 
     |████████████████████████████████| 182 kB 69.9 MB/s 


In [2]:
!gdown https://drive.google.com/uc?id=1n9LRtmKx-uTTIkCWHLgeTpPB_-bp7cn4

Downloading...
From: https://drive.google.com/uc?id=1n9LRtmKx-uTTIkCWHLgeTpPB_-bp7cn4
To: /content/pictograms.json
100% 7.78M/7.78M [00:00<00:00, 109MB/s]


In [ ]:
from tinydb import TinyDB,Query
import re
Pictogram = Query()
Keyword = Query()
db = TinyDB("pictograms.json")
picograms = db.table("pictogram")

def get_by_id(id_):
  pics = picograms.search(Pictogram._id == int(id_))
  if len(pics) > 0:
    return pics[0]
  return None

In [ ]:
!gdown https://drive.google.com/uc?id=1y2BYgwnlUg9jxHri8ggURKH9yai2rlQn

In [ ]:
vocab_new_sentences_path = "./vocab_new_sentences_path.txt"
nf = open(vocab_new_sentences_path,"r")
new_sentences = [l.rstrip() for l in nf.readlines()]
len(new_sentences)

2659

In [ ]:
import sqlite3
# db.close()
db = sqlite3.connect("file::memory:?cache=shared")

cur = db.cursor()
cur.execute("create table sentences(id INTEGER PRIMARY KEY AUTOINCREMENT,sentence text, telegraphic text)")
db.commit()
cur.close()

In [ ]:
cur = db.cursor()

cur.execute("DELETE FROM sentences")

db.commit()
cur.close()

cur = db.cursor()

cur.executemany("insert into sentences (sentence, telegraphic) values (?, ?)", [(item['Sentence'], item['Telegraphic']) for item in annotated])

cur.close()

db.commit()  

In [ ]:
def get_sentences(word,limit=50):
  sql = "SELECT ' ' || sentence || ' ' FROM sentences WHERE ' '|| sentence || ' ' LIKE '% {0} %' OR ' '|| telegraphic || ' ' LIKE '% {0} %' LIMIT {1}".format(word,limit)
  cur = db.cursor()
  cur.execute(sql)
  res = cur.fetchall()
  # res = execute_db(sql)
  return [a[0] for a in res if a[0].count(word) == 1]
  # return res

get_sentences("carne")

[' eu gosto de carne. ',
 ' eu já comi carne. ',
 ' eu gosto de comer carne. ',
 ' não gosto de comer carne. ',
 ' eu quero mais um pouco de carne. ']

In [ ]:
import openai, re

openai.api_key = "..."

def generate(prompt):
  response = openai.Completion.create(
    engine="davinci",
    prompt=prompt,
    temperature=0.7,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=2
  )
  res= response['choices'][0]['text'];

  # return res

  lines = res.splitlines()
  sentences = [lines[0]]
  for l in lines[1:]:
    parts = l.split(":")
    sentences.append(":".join(parts[1:]))
  sentences = [s for s in sentences if s != ""]

  return sentences

  # splited_sentences = re.split(r"\n\n", res) 
  # # sentences = ["".join(splited_sentences[0].split("[END]")).strip()]
  # sentences = []
  # for sen in splited_sentences[1:]:
  #   finds = re.findall("(?<=:)(.*)(?=\[END\])",sen)
  #   if len(finds) > 0:
  #     sentences.append(finds[0].strip())
  # return sentences


In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))


In [ ]:
exclude_categories = ["Perguntas","Algo a dizer"]

In [ ]:
import random
def get_example_sentences(chunk, min = 5,attempt = 0, max=6):
  sample = random.sample(chunk,5)
  example_sentences = []
  for word in sample:
    example_sentences = example_sentences + get_sentences(word,2)
  if len(example_sentences) < min:
    if attempt == 3:
      return example_sentences
    return get_example_sentences(chunk=chunk, min=min, attempt=attempt+1, max=max)
  return example_sentences[:6]


In [ ]:
import random
import math
# f = open("my_prompts.txt",'w')
cards = []
for item in vocabulary:
  
  if item['type'] == "CATEGORY":
    if item['label'] not in exclude_categories:
      # f.write("\n\n================================ {0} ================================\n\n".format(item['label']))
      
      for c in item['cards']:
        if c['type'] == "ELEMENT":
          words = [w.strip().lower() for w in c['label'].split(",")]
          cards = cards + words
      
len(cards)      

702

In [ ]:
random.shuffle(cards)
chunks_ = list(split(cards,math.floor(len(cards)/20)))
len(chunks_)

35

In [ ]:
from tqdm import tqdm

    
      # print(prompt)

100%|██████████| 35/35 [08:53<00:00, 15.24s/it]


In [ ]:
from tqdm import tqdm
for round in range(1,10):
  print(round)
  random.shuffle(cards)
  chunks_ = list(split(cards,math.floor(len(cards)/20)))
  for chunk in tqdm(chunks_):
    example_sentences = get_example_sentences(chunk)
    if len(example_sentences) > 0:
      prompt = "Generate at least twenty new unique Brazilian Portuguese sentences using the following words: {0}.\n\n".format(", ".join(['"{0}"'.format(w) for w in chunk]))

      for i, s in enumerate(example_sentences):
        prompt += "{0}: {1}\n".format(i+1, s.strip())
      prompt += "{0}:".format(i+2)
      
      sentences = list(set(generate(prompt)))
      true_new = [s for s in sentences if s not in all_sentences+new_sentences]
      # print("New:", len(true_new))
      new_sentences = new_sentences + true_new
  vocab_new_sentences_path = "/content/gdrive/MyDrive/Doutorado/Corpus construction/execution/files/13jul_vocab_new_sentences_path_round_{0}.txt".format(round)

  f = open(vocab_new_sentences_path,"w")
  f.writelines([l+"\n" for l in new_sentences])
  f.close()

1


100%|██████████| 35/35 [04:41<00:00,  8.05s/it]


2


100%|██████████| 35/35 [04:49<00:00,  8.27s/it]


3


...

100%|██████████| 35/35 [04:33<00:00,  7.83s/it]


8


100%|██████████| 35/35 [05:11<00:00,  8.90s/it]


9


100%|██████████| 35/35 [05:17<00:00,  9.07s/it]


In [ ]:
len(new_sentences)

3212

In [ ]:
new_sentences

['eu gosto de terça feira.',
 'eu não gosto de ônibus.',
 'eu gosto de falar.',
 'eu não gosto de presunto.',
 'eu gosto de calça.',
 'eu gosto de olhos.',
 'eu não gosto de barriga para cima.',
 'eu gosto de transparência.',
 'eu gosto de t o.',
 'eu gosto de penteado.',
 'eu não gosto de terça feira.',
 'eu quero comprar um bife de frango para meu filho.',
 'eu vou ser um bom pai para meu filho.',
 'eu quero amar meu irmão.',
 'eu vou ser um bom vizinho para os meus vizinhos.',
 'eu quero comprar um carro.',
 'eu quero tomar café na minha sala.',
 'eles têm casa.',
 'eles não têm casa.',
 'eu tenho casa.',
 'eles vão à praia amanhã.',
 'ele não tem casa.',
 'não quero nenhum.',
 'eu estou muito feliz!',
 'eu quero pizza.',
 'eu tenho cinco coelhos.',
 'você quer algo da geladeira?',
 'eu quero comer pizza hoje.',
 'eu quero camiseta curta.',
 'você quer camiseta curta?',
 'meu pai trabalha no Rio de Janeiro.',
 'eu sempre gostei do Rio de Janeiro.',
 'eu quero ir para o Rio de Janeir

In [ ]:
len(new_sentences) - 2217

442

In [ ]:
len(new_sentences)

2659

Rodada 1 (500 tokens): 439 sentenças por 1,5 dólares

Rodada 2 (1000 tokens): 455 sentenças por 2,03 dólares

Rodada 3 (500 tokens): 345 sentenças por 0,94 dólares

Rodada 4 (500 tokens): 488 sentenças por 1,18 dólares

Rodada 5 (500 tokens): 490 sentenças por 1,5 dólares

Rodada 6 (500 tokens): 442 sentenças por 1,39 dólares

In [ ]:
for chunk in chunks_[1:2]:
  example_sentences = get_example_sentences(chunk)
  prompt = "Generate at least twenty new unique Brazilian Portuguese sentences using the following words: {0}.\n\n".format(", ".join(['"{0}"'.format(w) for w in chunk]))

  for i, s in enumerate(example_sentences):
    prompt += "{0}: {1}\n".format(i+1, s.strip())
  prompt += "{0}:".format(i+2)
  print(prompt)

Generate at least twenty new unique Brazilian Portuguese sentences using the following words: "oculista", "um", "pizzaria", "padaria", "deles", "junho", "moletom", "voltar", "estrela", "rico", "girafa", "cuidado", "devagar", "botas", "trem", "difícil", "frio", "dormir", "quarta feira", "levantar", "curar".

1: eu tomei café da manhã devagar.
2: fale devagar por favor.
3: tenha cuidado com o fogo.
4: eu cozinhei com cuidado hoje.
5: eu vou entrar no trem.
6: o papai comprou um bola par mim no shopping.
7:
